In [ ]:
from pathlib import Path
from datetime import datetime, timedelta
import tempfile
import zipfile
import io

import pandas as pd
import sqlalchemy
import requests_cache
import requests
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
%matplotlib inline

In [ ]:
PATH_TO_RESULT_FILE = Path('./build/haringey-scenario-results.db').absolute()

LONDON_BOUNDARY_FILE_URL = 'https://files.datapress.com/london/dataset/statistical-gis-boundary-files-london/2016-10-03T13:52:28/statistical-gis-boundaries-london.zip'
WARD_SHAPE_FILE_PATH = Path('./statistical-gis-boundaries-london/ESRI/London_Ward.shp')
BUILD_FOLDER = Path('./build')
BUILD_FOLDER.mkdir(parents=True, exist_ok=True)

In [ ]:
disk_engine = sqlalchemy.create_engine('sqlite:///{}'.format(PATH_TO_RESULT_FILE))
requests_cache.install_cache((BUILD_FOLDER / 'cache').as_posix())

In [ ]:
def timedelta_from_iso_string(timedelta_as_string):
    t = datetime.strptime(timedelta_as_string,"PT%MM%SS")
    return timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)

assert timedelta_from_iso_string('PT19M43S') == timedelta(minutes=19) + timedelta(seconds=43)

## Read Metadata

In [ ]:
metadata = pd.read_sql_query('SELECT * FROM metadata', disk_engine, index_col='key', parse_dates=True)

In [ ]:
metadata

In [ ]:
metadata = dict(zip(metadata.index, [value[0] for value in metadata.values]))

In [ ]:
simulation_duration = timedelta_from_iso_string(metadata['durationOfSimulation'])

## People Activity

In [ ]:
activity = pd.read_sql_query('SELECT * FROM activity', disk_engine, index_col='timestamp', parse_dates=True)
activity.index = pd.to_datetime(activity.index * 1000 * 1000)
activity.index.name = 'datetime'
activity.head()

In [ ]:
sns.heatmap(pd.crosstab(activity.index, activity.value))
_ = plt.yticks([0, 144*0.25, 144*0.5, 144*0.75, 144], ["24:00", "18:00", "12:00", "06:00", "00:00"])
_ = plt.ylabel('time of day')
_ = plt.xlabel('activity')
_ = plt.title("Heatmap of people activity during the course of a day")

## Dwellings

In [ ]:
dwellings = pd.read_sql_query('SELECT * FROM dwellings', disk_engine, index_col='index')

In [ ]:
dwellings.head()

## Dwelling Temperature

In [ ]:
dwelling_temperature = pd.read_sql_query('SELECT * FROM temperature', disk_engine, index_col='timestamp', parse_dates=True)
dwelling_temperature.index = pd.to_datetime(dwelling_temperature.index * 1000 * 1000)
dwelling_temperature.index.name = 'datetime'
dwelling_temperature = dwelling_temperature.pivot(columns='id')
dwelling_temperature.head()

In [ ]:
_ = sns.violinplot(dwelling_temperature.mean(), orient='vertical')
_ = plt.title('Average temperature of dwellings')
_ = plt.ylabel('Average temperature [˚C]')

## Thermal Power

In [ ]:
thermal_power = pd.read_sql_query('SELECT * FROM thermalPower', disk_engine, index_col='timestamp', parse_dates=True)
thermal_power.index = pd.to_datetime(thermal_power.index * 1000 * 1000)
thermal_power.index.name = 'datetime'
thermal_power = thermal_power.pivot(columns='id')

In [ ]:
thermal_power.sum(axis=1).plot()

In [ ]:
dwelling_temperature.groupby(axis=1, by=lambda id: dwellings.loc[int(id[1]), 'ward']).mean().plot(figsize=(14, 7))

In [ ]:
ax = thermal_power\
    .groupby(axis=1, by=lambda id: dwellings.loc[int(id[1]), 'ward'])\
    .mean()\
    .plot(figsize=(14, 7), legend=None)
_ = plt.ylabel('average thermal power per household [W]')
_ = plt.title('Average of thermal power per household in different wards')
fig = ax.get_figure()
fig.savefig((BUILD_FOLDER / 'thermal_power_per_ward.png').as_posix())

In [ ]:
avg_power = thermal_power.groupby(axis=1, by=lambda id: dwellings.loc[int(id[1]), 'ward']).mean().mean()

In [ ]:
avg_power.name = 'average thermal power'

In [ ]:
r = requests.get(LONDON_BOUNDARY_FILE_URL)
z = zipfile.ZipFile(io.BytesIO(r.content))
with tempfile.TemporaryDirectory(prefix='london-boundary-files') as tmpdir:
    z.extractall(path=tmpdir)
    ward_file = Path(tmpdir) / WARD_SHAPE_FILE_PATH
    ward_data = gpd.read_file(ward_file.as_posix())
ward_data = ward_data[ward_data.BOROUGH == 'Haringey']

In [ ]:
import geopandasplotting as gpdplt # using not yet released geopandas code

In [ ]:
axes = gpdplt.plot_dataframe(ward_data.join(avg_power, on='NAME'), 
                             column='average thermal power',
                             categorical=False, 
                             linewidth=0.1, 
                             legend=True,
                             figsize=(10, 10))
legend = axes.get_legend()